In [34]:
import datetime
import re
import smtplib
from email.message import EmailMessage
from email.policy import SMTP
from docx import Document

# ── CONFIG ───────────────────────────────────────────────
VEHICLE_FILE     = r"D:\Coc_Intership_Files\old\Due Date Taxes for vehicle.docx"
LICENSE_FILE     = r"D:\Coc_Intership_Files\old\Driver_License_Record.docx"
MEDICAL_FILE     = r"D:\Coc_Intership_Files\old\Driver medical, eye, Fitness Certificate.docx"

SENDER_EMAIL      = "kamalkant23105@gmail.com"
SENDER_PASSWORD   = "aafe kpmj nubt wfdz"
RECEIVER_EMAIL    = "kamalkant23106@gmail.com"

VEHICLE_DAYS     = 30
LICENSE_DAYS     = 60
MEDICAL_DAYS     = 30
# ─────────────────────────────────────────────────────────

def _safe_cell(cells, idx) -> str:
    if idx >= len(cells):
        return ""
    return cells[idx].text.replace("\n", ", ").strip()

def normalise_dates(raw_text: str):
    raw_text = raw_text.replace(" ", "")
    return [p.split("AIP")[0].strip() for p in raw_text.split(",") if p.strip()]

# ── PARSE VEHICLE DOC ────────────────────────────────────
def parse_vehicle_doc(path: str):
    document = Document(path)
    rows = []
    current = {"Vehicle": "", "Road Tax Registration": "", "Fitness": "", "Permit": "", "Pollution Certificate": "", "Insurance": ""}

    for table in document.tables:
        got_any_vehicle = False
        for row in table.rows[1:]:
            cells = row.cells
            if len(cells) >= 2 and cells[1].text.strip():
                if got_any_vehicle:
                    rows.append(current.copy())
                current = {
                    "Vehicle": cells[1].text.strip(),
                    "Road Tax Registration": _safe_cell(cells, 2),
                    "Fitness": _safe_cell(cells, 3),
                    "Permit": _safe_cell(cells, 4),
                    "Pollution Certificate": _safe_cell(cells, 5),
                    "Insurance": _safe_cell(cells, 6)
                }
                got_any_vehicle = True
            else:
                current["Road Tax Registration"] += ", " + _safe_cell(cells, 2)
                current["Fitness"] += ", " + _safe_cell(cells, 3)
                current["Permit"] += ", " + _safe_cell(cells, 4)
                current["Pollution Certificate"] += ", " + _safe_cell(cells, 5)
                current["Insurance"] += ", " + _safe_cell(cells, 6)

        if got_any_vehicle:
            rows.append(current.copy())
    return rows

def check_vehicle_expiry(docs, days=VEHICLE_DAYS):
    today = datetime.date.today()
    future = today + datetime.timedelta(days=days)
    expiring = []
    expired = []

    for doc in docs:
        vehicle = doc["Vehicle"].strip()
        for col in ("Road Tax Registration", "Fitness", "Permit", "Pollution Certificate", "Insurance"):
            checked_dates = set()
            for date_txt in normalise_dates(doc[col]):
                try:
                    if date_txt in checked_dates:
                        continue
                    checked_dates.add(date_txt)
                    dt = datetime.datetime.strptime(date_txt, "%d-%m-%y").date()
                    if today <= dt <= future:
                        expiring.append((vehicle, col, dt.strftime("%d-%m-%Y")))
                    elif dt < today:
                        expired.append((vehicle, col, dt.strftime("%d-%m-%Y")))
                except ValueError:
                    continue
    return expiring, expired

# ── PARSE LICENSE DOC ───────────────────────────────────
def parse_license_doc(path: str):
    document = Document(path)
    records = []

    for table in document.tables:
        for row in table.rows[1:]:
            cells = row.cells
            if len(cells) < 4:
                continue
            records.append({
                "Driver": cells[1].text.strip(),
                "LicenceNo": cells[2].text.strip(),
                "DueDate": cells[3].text.strip(),
            })

    return records

def classify_license_expiry(licenses, days=LICENSE_DAYS):
    today = datetime.date.today()
    future = today + datetime.timedelta(days=days)

    expiring_soon = []
    already_expired = []

    for lic in licenses:
        try:
            dt = datetime.datetime.strptime(lic["DueDate"], "%d.%m.%y").date()
            formatted = dt.strftime("%d-%m-%Y")
            if today <= dt <= future:
                expiring_soon.append((lic["Driver"], lic["LicenceNo"], formatted))
            elif dt < today:
                already_expired.append((lic["Driver"], lic["LicenceNo"], formatted))
        except ValueError:
            continue
    return expiring_soon, already_expired

# ── PARSE MEDICAL DOC ───────────────────────────────────
def parse_medical_doc(file_path):
    document = Document(file_path)
    medical_data = []

    for table in document.tables:
        for row in table.rows[1:]:
            cells = row.cells
            try:
                record = {
                    'Driver': cells[1].text.strip(),
                    'Eye': cells[2].text.strip(),
                    'Fitness': cells[3].text.strip(),
                    'Audiometry': cells[4].text.strip()
                }
                medical_data.append(record)
            except IndexError:
                continue
    return medical_data

def classify_medical_records(records, days_before_expiry=MEDICAL_DAYS):
    today = datetime.date.today()
    alert_records = []
    expired_records = []

    for record in records:
        for test in ['Eye', 'Fitness', 'Audiometry']:
            try:
                test_date = datetime.datetime.strptime(record[test], "%d/%m/%Y").date()
                expiry_date = test_date + datetime.timedelta(days=365)
                alert_start = expiry_date - datetime.timedelta(days=days_before_expiry)

                if alert_start <= today <= expiry_date:
                    alert_records.append((record['Driver'], test, expiry_date.strftime("%d-%m-%Y")))
                elif expiry_date < today:
                    expired_records.append((record['Driver'], test, expiry_date.strftime("%d-%m-%Y")))
            except (ValueError, KeyError):
                continue

    return alert_records, expired_records

# ── EMAIL SENDER ─────────────────────────────────────────
def send_email(vehicle_alerts_combined, license_alerts_combined, medical_alerts_combined):
    vehicle_expiring, vehicle_expired = vehicle_alerts_combined
    license_expiring, license_expired = license_alerts_combined
    alert_records, expired_records = medical_alerts_combined

    msg = EmailMessage(policy=SMTP)
    msg['Subject'] = '🛑 Document Expiry Notification'
    msg['From'] = SENDER_EMAIL
    msg['To'] = RECEIVER_EMAIL

    body = "🛑 **Document Expiry Summary** 🛑\n\n"

    body += "🟡 **Vehicle Documents Expiring in Next 30 Days:**\n"
    body += "\n".join(f"• {v} ({typ}): {dt}" for v, typ, dt in vehicle_expiring) if vehicle_expiring else "None\n"

    body += "\n\n❌ **Vehicle Documents Already Expired:**\n"
    body += "\n".join(f"• {v} ({typ}) expired on: {dt}" for v, typ, dt in vehicle_expired) if vehicle_expired else "None\n"

    body += "\n\n🔵 **License Expiry (Next 60 Days):**\n"
    body += "\n".join(f"• {d} (License No: {lic}): {dt}" for d, lic, dt in license_expiring) if license_expiring else "None\n"

    body += "\n\n❌ **Already Expired Licenses:**\n"
    body += "\n".join(f"• {d} (License No: {lic}) expired on: {dt}" for d, lic, dt in license_expired) if license_expired else "None\n"

    body += "\n\n🟡🩺 **Medical Certificates Expiring (Next 30 Days):**\n"
    body += "\n".join(f"• {d} ({test}): {dt}" for d, test, dt in alert_records) if alert_records else "None\n"

    body += "\n\n❌ **Medical Certificates Already Expired:**\n"
    body += "\n".join(f"• {d} ({test}) expired on: {dt}" for d, test, dt in expired_records) if expired_records else "None\n"

    msg.set_content(body)

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
            smtp.login(SENDER_EMAIL, SENDER_PASSWORD)
            smtp.send_message(msg)
        print("✅ Email sent successfully.")
    except Exception as e:
        print(f"❌ Failed to send email: {e}")

# ── MAIN ────────────────────────────────────────────────
if __name__ == "__main__":
    vehicle_data = parse_vehicle_doc(VEHICLE_FILE)
    vehicle_alerts_combined = check_vehicle_expiry(vehicle_data)

    license_data = parse_license_doc(LICENSE_FILE)
    license_alerts_combined = classify_license_expiry(license_data)

    medical_data = parse_medical_doc(MEDICAL_FILE)
    medical_alerts_combined = classify_medical_records(medical_data)

    send_email(vehicle_alerts_combined, license_alerts_combined, medical_alerts_combined)


✅ Email sent successfully.
